In [1]:
from loguru import logger
import ndf.download
from ndf.datamining import datamining
import warnings
import sys
import pandas as pd

warnings.simplefilter(action='ignore', category=FutureWarning)

log_level = 'TRACE'
logger.add(sys.stderr, format="{time} {level} {message}", filter="my_module", level=log_level)
logger.add(sys.stdout, colorize=True, format="<green>{time}</green> <level>{message}</level>")
# logger.add("ndfdownload.log", rotation="30 MB", level=log_level)

2

In [2]:
import requests
import time

MAX_RETRIES = 4
TIME_TO_WAIT = 10

def download_manager(url):
    r = None
    # logger.info(f'Download settings: MAX_RETRIES:{MAX_RETRIES} | TIME_TO_WAIT:{TIME_TO_WAIT}')
    for _ in range(MAX_RETRIES):
        try:
            r = requests.get(url, verify=False)
        except requests.exceptions.RequestException as e:
            print(f'Download fail, {e}')
        if r:
            print(f'Download Ok')
            break
        else:
            wait = TIME_TO_WAIT * (_ + 1) if _ < 3 else TIME_TO_WAIT * (_ * 3)
            print(f'Download fail, retry in {wait} seconds...')
            time.sleep(wait)
    return r

In [3]:
# download_manager('https://www.tullettprebon.com/swap-execution-facility/daily-activity-summary.aspxa')

In [4]:
from datetime import datetime

a = datetime.now()
a.date()

datetime.date(2025, 3, 8)

In [5]:
# Logger settings

LOGGER_ENABLED = True

if not LOGGER_ENABLED:
    logger.remove()

In [6]:
d = ndf.download.download()
d.download_all()

2025-03-08 14:23:00.129 | INFO     | ndf.download:_download:55 - BGC - Download start >>>


2025-03-08T14:23:00.129947-0500 BGC - Download start >>>


2025-03-08 14:23:00.131 | DEBUG    | ndf.download:_download:58 - Try download bgc from http://dailyactprod.bgcsef.com/SEF/DailyAct/DailyAct_20250307.xlsx


2025-03-08T14:23:00.131488-0500 Try download bgc from http://dailyactprod.bgcsef.com/SEF/DailyAct/DailyAct_20250307.xlsx


2025-03-08 14:23:00.133 | INFO     | ndf.download:_download:80 - Loaded bgc DATA from cache /Users/cleiton/PycharmProjects/ndfutil/DATA/bgc_2025-03-08.pkl


...

2025-03-08T14:23:00.143874-0500 GFI - Download start >>>


2025-03-08 14:23:00.144 | DEBUG    | ndf.download:_download:58 - Try download gfi from https://www.gfigroup.com/doc/sef/marketdata/2025-03-07_daily_trade_data.xlsx


2025-03-08T14:23:00.144569-0500 Try download gfi from https://www.gfigroup.com/doc/sef/marketdata/2025-03-07_daily_trade_data.xlsx


2025-03-08 14:23:00.146 | INFO     | ndf.download:_download:80 - Loaded gfi DATA from cache /Users/cleiton/PycharmProjects/ndfutil/DATA/gfi_2025-03-08.pkl


2025-03-08T14:23:00.146892-0500 Loaded gfi DATA from cache /Users/cleiton/PycharmProjects/ndfutil/DATA/gfi_2025-03-08.pkl
bgc ok
tradition ok
prebontullet ok
gfi ok


In [7]:
mining = datamining()

In [8]:
d.download_bgc()

2025-03-08 14:23:00.154 | INFO     | ndf.download:_download:55 - BGC - Download start >>>


2025-03-08T14:23:00.154345-0500 BGC - Download start >>>


2025-03-08 14:23:00.155 | DEBUG    | ndf.download:_download:58 - Try download bgc from http://dailyactprod.bgcsef.com/SEF/DailyAct/DailyAct_20250307.xlsx


2025-03-08T14:23:00.155201-0500 Try download bgc from http://dailyactprod.bgcsef.com/SEF/DailyAct/DailyAct_20250307.xlsx


2025-03-08 14:23:00.157 | INFO     | ndf.download:_download:80 - Loaded bgc DATA from cache /Users/cleiton/PycharmProjects/ndfutil/DATA/bgc_2025-03-08.pkl


2025-03-08T14:23:00.157402-0500 Loaded bgc DATA from cache /Users/cleiton/PycharmProjects/ndfutil/DATA/bgc_2025-03-08.pkl


True

In [9]:
import joblib

df = None
df = joblib.load('/Users/cleiton/PycharmProjects/ndfutil/DATA/bgc_2025-03-08.pkl')

df


,BGC SEF PERMITTED AND REQUIRED INSTRUMENTS DAILY PRICE AND VOLUME,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8
0,Daily Activity Report for 20250307,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Asset Class,Currency,InstrumentDescription,Open,Low,High,Close,Block,Volume
2,CD,EUR,S42 XO 5Y. .ICC,293.25,293.25,296.75,296.75,NaN,40000000
3,CD,EUR,S42 XO 5Y. .LCH,293.5,293.5,297.75,297.75,NaN,70000000
4,CD,EUR,S42 EUR V1 - LCH,54.5,54.5,54.5,54.5,NaN,200000000
...,...,...,...,...,...,...,...,...,...
495,IR,USD,ZCI 20250307 20250311 5YUSD USD 1M8 0 5Y8 0 5...,2.45225,2.45225,2.4825,2.4825,NaN,190000000
496,IR,TRY,OIS 20250307 20250310 18MTRY TRY 3M5 179 3M5 ...,35.05,35.05,35.05,35.05,NaN,330000000
497,IR,USD,OPT 20250307 20260311 10YUSD USD 12M1 0 12M1 ...,667,666.5,667,666.5,NaN,437500000
498,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
# df.drop('Unnamed: 0', axis=1, inplace=True)
df.drop([0], axis=0, inplace=True)
df.reset_index(drop=True, inplace=True)
df.columns = df.iloc[0]
df.reset_index(drop=True, inplace=True)
df = df.rename(columns={'Contract Description': 'Description'})
df.drop(['Asset Class', 'Open', 'Low', 'High', 'Close', 'Block', 'Currency'], axis=1, inplace=True)
df = df[df['Volume'].notna()]
df

,InstrumentDescription,Volume
0,InstrumentDescription,Volume
1,S42 XO 5Y. .ICC,40000000
2,S42 XO 5Y. .LCH,70000000
3,S42 EUR V1 - LCH,200000000
4,S42 XO V2 - LCH,50000000
...,...,...
492,IRS 20250307 20250311 5YEUR EUR 12M2 167 6M1 ...,703900000
493,ZCI 20250307 20250311 5YUSD USD 1Y8 0 5Y8 0 5...,153000000
494,ZCI 20250307 20250311 5YUSD USD 1M8 0 5Y8 0 5...,190000000
495,OIS 20250307 20250310 18MTRY TRY 3M5 179 3M5 ...,330000000


In [11]:
df

,InstrumentDescription,Volume
0,InstrumentDescription,Volume
1,S42 XO 5Y. .ICC,40000000
2,S42 XO 5Y. .LCH,70000000
3,S42 EUR V1 - LCH,200000000
4,S42 XO V2 - LCH,50000000
...,...,...
492,IRS 20250307 20250311 5YEUR EUR 12M2 167 6M1 ...,703900000
493,ZCI 20250307 20250311 5YUSD USD 1Y8 0 5Y8 0 5...,153000000
494,ZCI 20250307 20250311 5YUSD USD 1M8 0 5Y8 0 5...,190000000
495,OIS 20250307 20250310 18MTRY TRY 3M5 179 3M5 ...,330000000


In [12]:
#
# DADOS ANTIGOS 
#

import joblib

df_antigo = joblib.load('/Users/cleiton/PycharmProjects/ndfutil/DATA/BGC_2023-04-28.pkl')

df_antigo

,Unnamed: 0,BGC SEF PERMITTED AND REQUIRED INSTRUMENTS DAILY PRICE AND VOLUME,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,"Copyright © 2013 BGC Derivative Market, L.P. ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,Trade Date,Asset Class,Currency,InstrumentDescription,Open,OpenType,Low,LowType,High,HighType,Close,CloseType,Block,EDRPVolume,Volume
4,NaN,2023-04-27,CD,EUR,S39 EUR 5Y. .LCH,84.5,TRANS,83.75,BID,84.5,BID,83.75,TRANS,NaN,NaN,75000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
455,NaN,2023-04-27,IR,EUR,OPT 20230427 20280502 12MEUR EUR,185.5,TRANS,185.5,TRANS,185.5,TRANS,185,TRANS,NaN,NaN,200000000
456,NaN,2023-04-27,IR,EUR,OPT 20230427 20280502 20YEUR EUR,2181,TRANS,2181,TRANS,2181,TRANS,2181,TRANS,NaN,NaN,50000000
457,NaN,2023-04-27,IR,EUR,OPT 20230427 20290502 10YEUR EUR,1457,TRANS,1457,TRANS,1457,TRANS,1457,TRANS,NaN,NaN,32000000
458,NaN,2023-04-27,IR,USD,OPT 20230427 20300501 20YUSD USD,2320,TRANS,2320,TRANS,2320,TRANS,2320,TRANS,NaN,NaN,25000000


In [13]:
df_antigo.drop('Unnamed: 0', axis=1, inplace=True)
df_antigo

,BGC SEF PERMITTED AND REQUIRED INSTRUMENTS DAILY PRICE AND VOLUME,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"Copyright © 2013 BGC Derivative Market, L.P. ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Trade Date,Asset Class,Currency,InstrumentDescription,Open,OpenType,Low,LowType,High,HighType,Close,CloseType,Block,EDRPVolume,Volume
4,2023-04-27,CD,EUR,S39 EUR 5Y. .LCH,84.5,TRANS,83.75,BID,84.5,BID,83.75,TRANS,NaN,NaN,75000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
455,2023-04-27,IR,EUR,OPT 20230427 20280502 12MEUR EUR,185.5,TRANS,185.5,TRANS,185.5,TRANS,185,TRANS,NaN,NaN,200000000
456,2023-04-27,IR,EUR,OPT 20230427 20280502 20YEUR EUR,2181,TRANS,2181,TRANS,2181,TRANS,2181,TRANS,NaN,NaN,50000000
457,2023-04-27,IR,EUR,OPT 20230427 20290502 10YEUR EUR,1457,TRANS,1457,TRANS,1457,TRANS,1457,TRANS,NaN,NaN,32000000
458,2023-04-27,IR,USD,OPT 20230427 20300501 20YUSD USD,2320,TRANS,2320,TRANS,2320,TRANS,2320,TRANS,NaN,NaN,25000000


In [14]:

df_antigo.drop([0, 1, 2], axis=0, inplace=True)
df_antigo.reset_index(drop=True, inplace=True)
df_antigo.columns = df_antigo.iloc[0]
df_antigo.reset_index(drop=True, inplace=True)
df_antigo = df_antigo.rename(columns={'Contract Description': 'Description'})
df_antigo.drop(['Asset Class', 'Open', 'Low', 'High', 'Close', 'Block', 'Currency'], axis=1, inplace=True)
df_antigo = df_antigo[df_antigo['Volume'].notna()]
df_antigo


,Trade Date,InstrumentDescription,OpenType,LowType,HighType,CloseType,EDRPVolume,Volume
0,Trade Date,InstrumentDescription,OpenType,LowType,HighType,CloseType,EDRPVolume,Volume
1,2023-04-27,S39 EUR 5Y. .LCH,TRANS,BID,BID,TRANS,NaN,75000000
2,2023-04-27,S39 XO 5Y. .LCH,TRANS,ASK,ASK,TRANS,NaN,10000000
3,2023-04-27,S39 EUR 5y. .ICC,TRANS,ASK,BID,TRANS,NaN,125000000
4,2023-04-27,S39 EUR V1 - ICC,TRANS,TRANS,TRANS,TRANS,NaN,300000000
...,...,...,...,...,...,...,...,...
451,2023-04-27,OPT 20230427 20280501 30YUSD USD,TRANS,TRANS,TRANS,TRANS,NaN,50000000
452,2023-04-27,OPT 20230427 20280502 12MEUR EUR,TRANS,TRANS,TRANS,TRANS,NaN,200000000
453,2023-04-27,OPT 20230427 20280502 20YEUR EUR,TRANS,TRANS,TRANS,TRANS,NaN,50000000
454,2023-04-27,OPT 20230427 20290502 10YEUR EUR,TRANS,TRANS,TRANS,TRANS,NaN,32000000


In [15]:
mining.bgc_calcs()

2025-03-08 14:23:00.236 | INFO     | ndf.datamining:bgc_calcs:500 - bgc calcs starting


2025-03-08T14:23:00.236581-0500 bgc calcs starting


2025-03-08 14:23:00.237 | DEBUG    | ndf.datamining:bgc_calcs:502 - Opening CACHE FILE /Users/cleiton/PycharmProjects/ndfutil/DATA/bgc_2025-03-08.pkl


2025-03-08T14:23:00.237710-0500 Opening CACHE FILE /Users/cleiton/PycharmProjects/ndfutil/DATA/bgc_2025-03-08.pkl


2025-03-08 14:23:00.244 | DEBUG    | ndf.datamining:bgc_calcs:547 - BGC before mining:
0                     Description    Volume
0  USDBRL NDF 07MAR2025 BILATERAL  31960000
1  USDBRL NDF 31MAR2025 BILATERAL  32000000


2025-03-08T14:23:00.244351-0500 BGC before mining:
0                     Description    Volume
0  USDBRL NDF 07MAR2025 BILATERAL  31960000
1  USDBRL NDF 31MAR2025 BILATERAL  32000000


2025-03-08 14:23:00.252 | DEBUG    | ndf.datamining:bgc_calcs:616 - BGC _summary ===>
       Total for human    Volume
Class                           
BMF                32M  32000000
BROKEN          31.96M  31960000
TOTAL           63.96M  63960000



2025-03-08T14:23:00.252273-0500 BGC _summary ===>
       Total for human    Volume
Class                           
BMF                32M  32000000
BROKEN          31.96M  31960000
TOTAL           63.96M  63960000



,Total for human,Volume
Class,,
BMF,32M,32000000
BROKEN,31.96M,31960000
TOTAL,63.96M,63960000


In [16]:
mining.tulletprebon_calcs()

2025-03-08 14:23:00.258 | INFO     | ndf.datamining:tulletprebon_calcs:177 - tulletprebon calcs starting


2025-03-08T14:23:00.258604-0500 tulletprebon calcs starting


2025-03-08 14:23:00.259 | DEBUG    | ndf.datamining:tulletprebon_calcs:179 - Opening CACHE FILE /Users/cleiton/PycharmProjects/ndfutil/DATA/tulletprebon_2025-03-08.pkl


2025-03-08T14:23:00.259757-0500 Opening CACHE FILE /Users/cleiton/PycharmProjects/ndfutil/DATA/tulletprebon_2025-03-08.pkl


2025-03-08 14:23:00.268 | DEBUG    | ndf.datamining:tulletprebon_calcs:198 - tullet before mining:
    Batch Date Asset Class       Tradeable Instrument  \
0   2025-03-07          CU     0D_NDF.USD.BRL.TOD_NDF   
1   2025-03-07          CU     0D_NDF.USD.BRL.TOM_NDF   
2   2025-03-07          CU   22D_NDF.USD.BRL.BMF1_NDF   
3   2025-03-07          CU            26D_NDF_USD_BRL   
4   2025-03-07          CU  300D_NDF.USD.BRL.300D_NDF   
5   2025-03-07          CU            35D_NDF_EUR_BRL   
6   2025-03-07          CU            35D_NDF_USD_BRL   
7   2025-03-07          CU           369D_NDF_USD_BRL   
8   2025-03-07          CU      3D_NDF.USD.BRL.3D_NDF   
9   2025-03-07          CU             4D_NDF_USD_BRL   
10  2025-03-07          CU   55D_NDF.USD.BRL.BMF2_NDF   
11  2025-03-07          CU             5D_NDF_USD_BRL   
12  2025-03-07          CU      6D_NDF.USD.BRL.6D_NDF   
13  2025-03-07          CU             6D_NDF_USD_BRL   
14  2025-03-07          CU    8D_NDF.USD.BRL.I

...

2025-03-08 14:23:00.269 | DEBUG    | ndf.datamining:tulletprebon_calcs:203 -     Batch Date Asset Class       Tradeable Instrument  \
0   2025-03-07          CU     0D_NDF.USD.BRL.TOD_NDF   
1   2025-03-07          CU     0D_NDF.USD.BRL.TOM_NDF   
2   2025-03-07          CU   22D_NDF.USD.BRL.BMF1_NDF   
3   2025-03-07          CU            26D_NDF_USD_BRL   
4   2025-03-07          CU  300D_NDF.USD.BRL.300D_NDF   
5   2025-03-07          CU            35D_NDF_EUR_BRL   
6   2025-03-07          CU            35D_NDF_USD_BRL   
7   2025-03-07          CU           369D_NDF_USD_BRL   
8   2025-03-07          CU      3D_NDF.USD.BRL.3D_NDF   
9   2025-03-07          CU             4D_NDF_USD_BRL   
10  2025-03-07          CU   55D_NDF.USD.BRL.BMF2_NDF   
11  2025-03-07          CU             5D_NDF_USD_BRL   
12  2025-03-07          CU      6D_NDF.USD.BRL.6D_NDF   
13  2025-03-07          CU             6D_NDF_USD_BRL   
14  2025-03-07          CU    8D_NDF.USD.BRL.IMM1_NDF   
15  2025-03

2025-03-08T14:23:00.269179-0500     Batch Date Asset Class       Tradeable Instrument  \
0   2025-03-07          CU     0D_NDF.USD.BRL.TOD_NDF   
1   2025-03-07          CU     0D_NDF.USD.BRL.TOM_NDF   
2   2025-03-07          CU   22D_NDF.USD.BRL.BMF1_NDF   
3   2025-03-07          CU            26D_NDF_USD_BRL   
4   2025-03-07          CU  300D_NDF.USD.BRL.300D_NDF   
5   2025-03-07          CU            35D_NDF_EUR_BRL   
6   2025-03-07          CU            35D_NDF_USD_BRL   
7   2025-03-07          CU           369D_NDF_USD_BRL   
8   2025-03-07          CU      3D_NDF.USD.BRL.3D_NDF   
9   2025-03-07          CU             4D_NDF_USD_BRL   
10  2025-03-07          CU   55D_NDF.USD.BRL.BMF2_NDF   
11  2025-03-07          CU             5D_NDF_USD_BRL   
12  2025-03-07          CU      6D_NDF.USD.BRL.6D_NDF   
13  2025-03-07          CU             6D_NDF_USD_BRL   
14  2025-03-07          CU    8D_NDF.USD.BRL.IMM1_NDF   
15  2025-03-07          CU            96D_NDF_USD_BRL   

2025-03-08 14:23:00.287 | DEBUG    | ndf.datamining:tulletprebon_calcs:357 - TULLETPREBON Summary ===>
        Total for human        Volume
Class                                
BMF1              1.25B  1.247200e+09
BMF2               220M  2.200000e+08
BROKEN           117.1M  1.171000e+08
LONGER             110M  1.100000e+08
PTAX              77.5M  7.750000e+07
TOMPTAX             73M  7.300000e+07
TOTAL              2.6B  2.596050e+09
XP              751.25M  7.512500e+08



2025-03-08T14:23:00.287243-0500 TULLETPREBON Summary ===>
        Total for human        Volume
Class                                
BMF1              1.25B  1.247200e+09
BMF2               220M  2.200000e+08
BROKEN           117.1M  1.171000e+08
LONGER             110M  1.100000e+08
PTAX              77.5M  7.750000e+07
TOMPTAX             73M  7.300000e+07
TOTAL              2.6B  2.596050e+09
XP              751.25M  7.512500e+08



,Total for human,Volume
Class,,
BMF1,1.25B,1.247200e+09
BMF2,220M,2.200000e+08
BROKEN,117.1M,1.171000e+08
LONGER,110M,1.100000e+08
PTAX,77.5M,7.750000e+07
TOMPTAX,73M,7.300000e+07
TOTAL,2.6B,2.596050e+09
XP,751.25M,7.512500e+08


In [17]:
mining.tradition_calcs()

2025-03-08 14:23:00.294 | INFO     | ndf.datamining:tradition_calcs:40 - tradition calcs starting


2025-03-08T14:23:00.294421-0500 tradition calcs starting


2025-03-08 14:23:00.295 | DEBUG    | ndf.datamining:tradition_calcs:42 - Opening CACHE FILE /Users/cleiton/PycharmProjects/ndfutil/DATA/tradition_2025-03-08.pkl


2025-03-08T14:23:00.295559-0500 Opening CACHE FILE /Users/cleiton/PycharmProjects/ndfutil/DATA/tradition_2025-03-08.pkl


2025-03-08 14:23:00.558 | DEBUG    | ndf.datamining:tradition_calcs:167 - TRADITION Summary ===>
       Total for human      Volume
Class                             
BROKEN           1.75B  1745750020
TOTAL            1.75B  1745750020



2025-03-08T14:23:00.558413-0500 TRADITION Summary ===>
       Total for human      Volume
Class                             
BROKEN           1.75B  1745750020
TOTAL            1.75B  1745750020



,Total for human,Volume
Class,,
BROKEN,1.75B,1745750020
TOTAL,1.75B,1745750020


In [18]:
import joblib

df = None
df = joblib.load('/Users/cleiton/PycharmProjects/ndfutil/DATA/gfi_2025-03-08.pkl')

df


,GFI SEF PERMITTED AND REQUIRED INSTRUMENTS DAILY PRICE AND VOLUME,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8
0,Daily Activity Report for 20250307,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Asset Class,Currency,InstrumentDescription,Open,Low,High,Close,Block,Volume
2,CD,USD,HY43 V1 5y - ICC,106.8925,106.8925,106.8925,106.8925,NaN,10000000
3,CD,EUR,S42 Eur 5y. .ICC,54.75,54.75,55.125,55.125,NaN,75000000
4,CD,EUR,S42 Xover 5y. .LCH,294,294,294,294,NaN,400000000
...,...,...,...,...,...,...,...,...,...
214,IR,CLP,CLP/CAMARA Interest Rate Swap CLP/CAMARA 1Y,5.055,5.055,5.055,5.055,NaN,50000000000
215,IR,CLP,CLP/CAMARA Interest Rate Swap CLP/CAMARA 3M,5.045,5.045,5.045,5.045,NaN,375000000000
216,IR,COP,COPNon DeliverableIRS COP IBR 6M,8.725,8.725,8.725,8.725,NaN,261000000000
217,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
df.drop([0], axis=0, inplace=True)
df.reset_index(drop=True, inplace=True)
df.columns = df.iloc[0]
df.reset_index(drop=True, inplace=True)
df = df.rename(columns={'InstrumentDescription': 'Description'})
df.drop(['Asset Class', 'Open', 'Low', 'High', 'Close', 'Block', 'Currency'], axis=1, inplace=True)
df = df[df['Volume'].notna()]
df

,Description,Volume
0,InstrumentDescription,Volume
1,HY43 V1 5y - ICC,10000000
2,S42 Eur 5y. .ICC,75000000
3,S42 Xover 5y. .LCH,400000000
4,S42 Eur 5y. .LCH,75000000
...,...,...
211,COPNon DeliverableIRS COP IBR 9M,119000000000
212,COPNon DeliverableIRS COP IBR 12M,91000000000
213,CLP/CAMARA Interest Rate Swap CLP/CAMARA 1Y,50000000000
214,CLP/CAMARA Interest Rate Swap CLP/CAMARA 3M,375000000000


In [20]:
mining.gfi_calcs()

2025-03-08 14:23:00.597 | INFO     | ndf.datamining:gfi_calcs:367 - gfi calcs starting


2025-03-08T14:23:00.597471-0500 gfi calcs starting


2025-03-08 14:23:00.598 | DEBUG    | ndf.datamining:gfi_calcs:369 - Opening CACHE FILE /Users/cleiton/PycharmProjects/ndfutil/DATA/gfi_2025-03-08.pkl


2025-03-08T14:23:00.598751-0500 Opening CACHE FILE /Users/cleiton/PycharmProjects/ndfutil/DATA/gfi_2025-03-08.pkl


2025-03-08 14:23:00.603 | DEBUG    | ndf.datamining:gfi_calcs:421 - GFI before mining:
0                     Description     Volume
0  USDBRL NDF 17MAR2025 BILATERAL  176000000
1  USDBRL NDF 10MAR2025 BILATERAL  286000000
2  USDBRL NDF 07MAR2025 BILATERAL  111000000
3  USDBRL NDF 30SEP2025 BILATERAL   25000000
4  USDBRL NDF 30MAY2025 BILATERAL  176000000
5  USDBRL NDF 11MAR2025 BILATERAL   44000000
6  USDBRL NDF 20MAR2025 BILATERAL   76000000
7  USDBRL NDF 31MAR2025 BILATERAL  664300000


2025-03-08T14:23:00.603781-0500 GFI before mining:
0                     Description     Volume
0  USDBRL NDF 17MAR2025 BILATERAL  176000000
1  USDBRL NDF 10MAR2025 BILATERAL  286000000
2  USDBRL NDF 07MAR2025 BILATERAL  111000000
3  USDBRL NDF 30SEP2025 BILATERAL   25000000
4  USDBRL NDF 30MAY2025 BILATERAL  176000000
5  USDBRL NDF 11MAR2025 BILATERAL   44000000
6  USDBRL NDF 20MAR2025 BILATERAL   76000000
7  USDBRL NDF 31MAR2025 BILATERAL  664300000


2025-03-08 14:23:00.638 | DEBUG    | ndf.datamining:gfi_calcs:490 - GFI _summary ===>
            Total for human      Volume
Class                                  
BMF                  664.3M   664300000
BMF 10/2025             25M    25000000
BMF3                   176M   176000000
BROKEN                 693M   693000000
TOTAL                 1.56B  1558300000



2025-03-08T14:23:00.638719-0500 GFI _summary ===>
            Total for human      Volume
Class                                  
BMF                  664.3M   664300000
BMF 10/2025             25M    25000000
BMF3                   176M   176000000
BROKEN                 693M   693000000
TOTAL                 1.56B  1558300000



,Total for human,Volume
Class,,
BMF,664.3M,664300000
BMF 10/2025,25M,25000000
BMF3,176M,176000000
BROKEN,693M,693000000
TOTAL,1.56B,1558300000


In [21]:
tulletprebon['source'] = 'TulletPrebon'
tradition['source'] = 'Tradition'
bgc['source'] = 'bgc'
gfi['source'] = 'gfi'

NameError: name 'tulletprebon' is not defined

In [ ]:
bgc

In [ ]:
df_result = pd.concat([tulletprebon, tradition, bgc, gfi])
df_result

In [ ]:
df_totals = df_result.query('Class == "TOTAL"')
df_totals

In [ ]:
df_totals.groupby(['Class']).sum()

In [ ]:
from numerize import numerize

market_total = df_totals.sum()['Volume']
market_total = numerize.numerize(float(market_total))
market_total